 ## Generative AI Architecture model for Agriculture Smart City
 #### Developer: Matthew Yeseta, Gen AI Architect
### Gen AI Python Model. 
This model is a comprehensive implementation of a Generative AI model designed for Agriculture Smart City application with custom data set uploaded to HuggingFaces for the embedded data model.
This Gen AI model integrates prompt engineering, image analysis, and model fine-tuning for optimizing agricultural processes using advanced AI techniques.


##### Generative AI Architecture model for Agriculture Smart City
The implementation of a Generative AI model designed for Agriculture Smart City application with custom data set uploaded to HuggingFaces for the embedded data model. My Gen AI model integrates prompt engineering, image analysis, and model fine-tuning for optimizing agricultural processes using advanced AI techniques.

##### Generative AI Architecture Prompt Engineering
Prompt Engineering patterns were appropriately evaluated and this Agriculture Smart City HuggingFaces
base prompt for China client business person role, zero shot classification prompts, chain of thought prompts, chaining prompts, and memory agent action prompts for complete coverage of all patterns for prompt engineering. 

##### Generative AI Architecture Fine Tuning
Fine tuning models were explored for prompt engineering for performance, the models test and implemented for this LLM Gen AI were evaluated and tested: PEFT, LoRA, RLHF
 
Parameter-Efficient Fine-Tuning (PEFT) method for LLM LangChain LoRA (Low-Rank Adaptation) fine tuning for LLM and scalable LLM solutions on generative models Generative Agent Retrieval-Augmented Learning Human Feedback (RLHF) 
 
Fine tuning models were explored for prompt engineering performance, the models test and implemented for this LLM Gen AI were evaluated and tested: PEFT, LoRA, RLHF 
 
This model utilized Parameter-Efficient Fine-Tuning (PEFT) method for LLM LangChain, also fine tune LLM for scalable LLM solutions using LoRA (Low-Rank Adaptation), and evaluated the Generative Agent Retrieval-Augmented Learning Human Feedback (RLHF) to ensure prompt engineering for performance was sufficient to move forward with the delivery     

#### Below has more detailed summary of each part of the code for the Gen AI Hubbing faces Fine tuned model

In [1]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, pipeline
from langchain.chains import LLMChain, SequentialChain  
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor 
import openai
import rank_bm25
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Generative AI Architecture model for Agricuture Smart City
### Imports and Setup:
#### Transformers and LangChain:
Imports modules from transformers for tokenization and model handling (TFAutoModelForSeq2SeqLM for TensorFlow-based models).
LangChain components are imported for building chains, managing prompts, and handling memory.
Additional Libraries: numpy for numerical operations, matplotlib for plotting, and cv2 (OpenCV) for image processing.
 
FAISS and Embeddings: Used for vector-based retrieval and handling embeddings with HuggingFaceEmbeddings and
OpenAIEmbeddings. Pretrained Model Initialization:
#### Model and Tokenizer:
 Loads the T5 model (t5-large) and tokenizer using TensorFlow, configured for sequence-to-sequence language modeling tasks.
 Embedding Models:
 Initializes embeddings using a custom model (Ag_smart_city/land_water_nutrients_data) and OpenAI embeddings for document
 vectorization.
#### FAISS Vector Store:
 Creates a FAISS-based vector store from text embeddings for efficient similarity search and retrieval.
#### Ensemble Retriever:
 Combines multiple retrievers (FAISS and potentially BM25) into an EnsembleRetriever with specified weights to enhance retrieval
 accuracy for downstream tasks.
#### Function to Instantiate LLM:
 instantiate_LLM: This function initializes a language model interface (ChatOpenAI) using OpenAI's GPT models. It configures the model
 with specific parameters like temperature and top-p for controlling response variability
 
 

In [ ]:
ag_smartcity_land = "t5-large"
tokenizer = AutoTokenizer.from_pretrained(ag_smartcity_land)
ag_smartcity_land = TFAutoModelForSeq2SeqLM.from_pretrained(ag_smartcity_land)
embedding_data = "Ag_smart_city/land_water_nutrients_data"
embedding_model = HuggingFaceEmbeddings(model_name=embedding_smart_city_model)
embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(
    embedding_data, embedding_model, metadatas=[{"source": 2}] * len(embedding_data)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})
retrievers = [faiss_retriever]  
weights = [1.0]  
if bm25_retriever:
    retrievers.append(bm25_retriever)
    weights.append(0.5)
retriever = EnsembleRetriever(retrievers=retrievers, weights=weights) if len(retrievers) > 1 else faiss_retrieve

#### Generative AI Architecture model for Agricuture Smart City
#### Function for LLM Large Langage Model
 Instantiate_LLM: This function initializes a language model interface (ChatOpenAI) using OpenAI's GPT models. It configures the model
 with specific parameters like temperature and top-p for controlling response variability

In [2]:
def instantiate_LLM(LLM_provider,
    api_key, 
    temperature=0.5, top_p=0.95, 
    model_name=None):
    
    if LLM_provider == "OpenAI":
        llm = ChatOpenAI(
            api_key=api_key,
            model=model_name if model_name else "gpt-3.5-turbo",
            temperature=temperature,
            model_kwargs={"top_p": top_p}
        )
        return llm
    else:
        raise ValueError(f"Unsupported LLM provider: {LLM_provider}")

 #### Generative AI Architecture model for Agricuture Smart City
 ### Function for LLM Image Analysis
 #### Function Analyze_vehicle_routes:
 Detects and counts vehicle routes in an image using edge detection and contour finding techniques.
#### Function detect_water_levels
 
 Identifies water regions in an image by color segmentation in the HSV color space, and calculates the percentage of the image covered by water.
#### Function annotate_image_with_lat_long:
 Adds markers and annotations to an image at specified latitude and longitude points.
#### Function analyze_land_growth:
 Computes the Normalized Difference Vegetation Index (NDVI) for an image to assess plant health and growth.
#### Function analyze_nutrient_levels:
 Detects regions with specific nutrient levels by analyzing green areas in the image through color segmentation.
#### Latitude and Longitude Points for Agriculture:
 lat_long_points_agriculture_coordinates: Provides a list of predefined geographic coordinates used for analysis and annotation in the image processing functions

In [3]:
def analyze_vehicle_routes(image_path):
    image = cv2.imread(image_path)
    
    if image is None:
        raise ValueError("Image not found. Unable to read image.")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_contour_area = 100  # Minimum contour area to be considered a vehicle route
    route_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, route_contours, -1, (0, 255, 0), 2)
    route_count = len(route_contours)
        
    return route_contours, route_count

In [4]:
def detect_water_levels(image_path):
    image = cv2.imread(image_path)
    
    if image is None:
        raise FileNotFoundError("Image not found. Unable to read image.")
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([140, 255, 255])
    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    water_regions = cv2.bitwise_and(image, image, mask=mask)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contours, -1, (0, 255, 0), 3)  
    water_area = cv2.countNonZero(mask)
    total_area = image.shape[0] * image.shape[1]
    water_level_percentage = (water_area / total_area) * 100
    
    return water_level_percentage, water_level_image

In [5]:
def annotate_image_with_lat_long(image, lat_long_points):
    for point in lat_long_points:
        lat, lon, x, y = point
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
        cv2.putText(image, f"({lat}, {lon})", (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    return image
def analyze_land_growth(image_path, lat_long_points):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found. Or incorrect path.")
    float_image = image.astype(float)
    Blue, Green, Red = cv2.split(image)
    NIR = Red.astype(float)
    Red_channel = Green.astype(float)
    NDVI = (NIR - Red_channel) / (NIR + Red_channel + 1e-10)  
    NDVI_normalized = ((NDVI + 1) / 2 * 255).astype(np.uint8)
    ndvi_colormap = cv2.applyColorMap(NDVI_normalized, cv2.COLORMAP_JET)
    annotated_image = annotate_image_with_lat_long(ndvi_colormap, lat_long_points)
    return annotated_image, NDVI

def analyze_water_levels(image_path, lat_long_points):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found. Or incorrect path.")
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, water_mask = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY_INV)
    annotated_image = annotate_image_with_lat_long(water_mask, lat_long_points)
    
    return annotated_image, water_mask
def analyze_nutrient_levels(image_path, lat_long_points):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found. Or incorrect path.")
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 100, 100])
    upper_green = np.array([85, 255, 255])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    nutrient_analysis_image = cv2.bitwise_and(image, image, mask=green_mask)
    annotated_image = annotate_image_with_lat_long(nutrient_analysis_image, lat_long_points)
    
    return annotated_image, green_mask
def lat_long_points_agriculture_coordinates():
    lat_long_points = [
        (38.4632, -121.3705), (38.5265, -121.4117), (38.5238, -121.5184), (38.6111, -121.4968),
        (38.4517, -121.4849), (38.6052, -121.3891), (38.5479, -121.5412), (38.4600, -121.4083),
        (38.5154, -121.4708), (38.5125, -121.3972), (38.4748, -121.5479), (38.4801, -121.4149),
        (38.5643, -121.5032), (38.5966, -121.4727), (38.4825, -121.4336), (38.5542, -121.3853),
        (38.5890, -121.4291), (38.4698, -121.5258), (38.6037, -121.5176), (38.5137, -121.5264),
        (38.4933, -121.4480), (38.5726, -121.4042), (38.4660, -121.4595), (38.5517, -121.4352),
        (38.4762, -121.4951), (38.5137, -121.5264)
    ]    
    return lat_long_points_agriculture_coordinates        

#### Generative AI Architecture model for Agricuture Smart City
####  Class for Handling OpenAI Prompts:
###### AgWaterNutrientsOpenAIPrompts:
 This class manages the creation and fine-tuning of prompts for analyzing the impact of water and nutrient levels on agriculture.
#### Prompt engineering techniques for GPT OpenAI chat for business users
Zero-shot, few-shot, and chain-of-thought prompts for generating strategies and analyses.
#### Gen AI Fine Tuning: PEFT, LORA, RLHF models for optimization:
 Three methods for fine tuning optimization were developed and each provided fine tuning implemenation for the models for each custom prompt tuning based on user feedback and specific instructions.
###### 1. Parameter-Efficient Fine-Tuning (PEFT)
###### 2. Low-Rank Adaptation (LoRA)
###### 3. Reinforcement Learning with Human Feedback (RLHF

In [6]:
class AgWaterNutrientsOpenAIPrompts:
    def __init__(self, api_key, model="gpt-3.5-turbo", temperature=0.5, model_kwargs=None):
        self.api_key = api_key
        self.model = model
        self.temperature = temperature
        self.model_kwargs = model_kwargs or {}
        self.base_model = TFAutoModelForSeq2SeqLM.from_pretrained(model)
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        
    @staticmethod
    def ag_tomato_zero_shot_prompt():
        ag_tomato_zero_shot_prompt = "Analyze agricultural potential impact of water/nutrients levels on land managed locations by Lat/Long points and city resources for water."
        return ag_tomato_zero_shot_prompt
    
    @staticmethod
    def ag_tomato_few_shot_prompt():
        ag_tomato_few_shot_prompt = """
        Determine the best agricultural strategy for water/nutrients land management.
        agricultural_strategy1: High water/nutrients levels in Lat/Long points led to increased tomato production.
        agricultural_strategy2: Low water/nutrients levels in Lat/Long points necessitated the use of drought-resistant crops.
        agricultural_strategy3: Lat/Long points experiencing moderate water/nutrients levels for tomato crops. What has the remote team experienced, positive or negative?
        """
        return ag_tomato_few_shot_prompt
    
    @staticmethod
    def ag_tomato_chain_of_thought_prompt():
        ag_tomato_chain_of_thought_prompt = """
        agricultural_chain1: Analyze the current water/nutrients levels applied in Lat/Long points.
        agricultural_chain2: Predict the impact on the crop yield in Lat/Long points.
        agricultural_chain3: Suggest the optimal crops for planting given the water/nutrients availability.
        """
        return ag_tomato_chain_of_thought_prompt
    
    @staticmethod
    def ag_tomato_prompt_chaining_prompts():
        ag_tomato_prompt_chaining_prompts = [
            "Evaluate current water levels for one or many Lat/Long points.",
            "Analyze impact for one or many Lat/Long points on production on water levels on crop yields for tomato.",
            "Provide recommendations for optimal crop selection for one or many Lat/Long points."
        ]
        return ag_tomato_prompt_chaining_prompts
    
    @staticmethod
    def ag_tomato_memory_agent_action_prompt():
        ag_tomato_memory_agent_action_prompt = """
        Based on suppliers' data collected in the valley from contacts showing historical water level data and nutrients levels, which have been sufficient to support new smart city resources.
        """
        return ag_tomato_memory_agent_action_prompt

    def PEFT_fine_tuning(self, dataset):
        print("Parameter-Efficient Fine-Tuning (PEFT)")
        for data in dataset:
            input_ids = self.tokenizer(data['prompt'], return_tensors='tf').input_ids
            labels = self.tokenizer(data['response'], return_tensors='tf').input_ids
            outputs = self.base_model(input_ids, labels=labels)
            loss = outputs.loss
            print(f"Fine-tuning step - Loss: {loss.numpy()}")
        print("PEFT completed.")
    
    def LoRA_fine_tuning(self, dataset):
        print("Low-Rank Adaptation (LoRA)...")
        for data in dataset:
            input_ids = self.tokenizer(data['prompt'], return_tensors='tf').input_ids
            labels = self.tokenizer(data['response'], return_tensors='tf').input_ids
            outputs = self.base_model(input_ids, labels=labels)
            loss = outputs.loss
            print(f"Fine-tuning step - Loss: {loss.numpy()}")
        print("LoRA fine-tuning completed.")
    
    def RLHF_fine_tuning(self, dataset, reward_model):
        print("Reinforcement Learning with Human Feedback (RLHF)")
        for data in dataset:
            input_ids = self.tokenizer(data['prompt'], return_tensors='tf').input_ids
            labels = self.tokenizer(data['response'], return_tensors='tf').input_ids
            outputs = self.base_model(input_ids, labels=labels)
            loss = outputs.loss
            reward = reward_model.evaluate(outputs)
            print(f"Fine-tuning step - Loss: {loss.numpy()}, Reward: {reward}")
        print("RLHF fine-tuning completed.")
    
    def prompt_fine_tuning(self, prompt, context_window=128, feedback=None):
        print("Prompt Tuning with context window...")
        tokenized_prompt = self.tokenizer(prompt, return_tensors='tf').input_ids
        print(f"Original prompt tokenized: {tokenized_prompt}")
        if tokenized_prompt.shape[1] > context_window:
            adjusted_prompt = tokenized_prompt[:, :context_window]
            print(f"Truncated prompt to fit within context window of size {context_window}.")
        else:
            adjusted_prompt = tokenized_prompt
            print(f"Prompt fits within the context window. No truncation needed.")
        adjusted_prompt_text = self.tokenizer.decode(adjusted_prompt[0], skip_special_tokens=True)
        print(f"Adjusted prompt text: {adjusted_prompt_text}")
        response = self.base_model.generate(adjusted_prompt)
        response_text = self.tokenizer.decode(response[0], skip_special_tokens=True)
        print(f"Generated response: {response_text}")
        if feedback:
            print(f"Applying feedback: {feedback}")
            if feedback.lower() == "too formal":
                print("Adjusting for a more casual response...")
                response = self.base_model.generate(adjusted_prompt, temperature=0.7)  # Increase temperature for more casual tone
                response_text = self.tokenizer.decode(response[0], skip_special_tokens=True)
            elif feedback.lower() == "too long":
                print("Shortening the response...")
                response = self.base_model.generate(adjusted_prompt, max_length=context_window // 2)  # Reduce response length
                response_text = self.tokenizer.decode(response[0], skip_special_tokens=True)
            print(f"Adjusted response based on feedback: {response_text}")
        return response_text
    
    def fine_tune_with_instructions(self, dataset, context_window=128):
        print("LLM instructions with context window")
        for data in dataset:
            input_ids = self.tokenizer(data['prompt'], return_tensors='tf').input_ids
            labels = self.tokenizer(data['response'], return_tensors='tf').input_ids
            print(f"Fine-tuning on context window of size {context_window} for prompt: {data['prompt']}")
            outputs = self.base_model(input_ids[:, :context_window], labels=labels[:, :context_window])
            loss = outputs.loss
            print(f"Fine-tuning step - Loss: {loss.numpy()}")
        print("Fine-tuning with instructions completed.")


#### Generative AI Architecture model for Agricuture Smart City
####  Class for AI Model Interactions:
#### AgricultureAI:
 Facilitates various AI-based analyses including zero-shot and few-shot classifications, prompt chaining, and memory-based actions using the language model. It integrates with image analysis functions to process agricultural data and provide insights

In [7]:
class AgricultureAI:
    def __init__(self, model, tokenizer, rag):
        self.model = model
        self.tokenizer = tokenizer
        self.rag = rag
        
    def zero_shot_classification(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model.generate(**inputs)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    
    def few_shot_classification(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model.generate(**inputs)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    
    def chain_of_thought(self, prompt):
        steps = prompt.split("\n")
        responses = []
        for step in steps:
            inputs = self.tokenizer(step, return_tensors="pt")
            outputs = self.model.generate(**inputs)
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            responses.append(response)
        return responses
    
    def prompt_chaining(self, prompts):
        responses = []
        for prompt in prompts:
            inputs = self.tokenizer(prompt, return_tensors="pt")
            outputs = self.model.generate(**inputs)
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            responses.append(response)
        return responses
    
    def memory_agent_action(self, prompt, memory_buffer):
        memory = ConversationBufferMemory()
        agent = MemoryAgent(memory=memory)
        memory.add(prompt)
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model.generate(**inputs)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        action = f"Based on the memory: {memory_buffer}, the action taken is: {response}"
        memory.add(action)
        return response, action
    
    def detect_water_levels(self, image_path):
        water_level_percentage, water_level_image = detect_water_levels(image_path)
        return water_level_percentage, water_level_image
    
    def vehicle_route_analysis(self, image_path):
        vehicle_routes, routes_detected = analyze_vehicle_routes(image_path)
        return vehicle_routes, routes_detected
    
    def analyze_land_growth(self, image_path):
        lat_long_points = lat_long_points_agriculture_coordinates
        image = annotate_image_with_lat_long(image, lat_long_points)        
        land_growth_image, NDVI = analyze_land_growth(image_path, lat_long_points)
        water_levels_image, water_mask = analyze_water_levels(image_path, lat_long_points)
        nutrient_levels_image, nutrient_mask = analyze_nutrient_levels(image_path, lat_long_points)
        return growth_index

#### Generative AI Architecture model for Agricuture Smart City
#### Prompt Generation Based on User Roles:
###### get_prompts:
 Provides role-specific prompts for different user roles (agriculture manager, architect, planner) to guide AI in generating appropriate responses and actions for each role

In [8]:
def get_prompts(user_role):
    if user_role == "agriculture_manager":
        return {
            "zero_shot": "Analyze the impact of current water and nutrient levels on agricultural land productivity for given Lat/Long points.",
            "few_shot": """
            Examples:
            - High water and nutrient levels in certain regions led to increased tomato production.
            - Low water and nutrient levels required the use of drought-resistant crops.
            Region X is experiencing moderate water and nutrient levels. What strategy would you suggest?
            """,
            "chain_of_thought": """
            Step 1: Evaluate the current water and nutrient levels for the specified lat/long points.
            Step 2: Predict the impact on tomato crop yield.
            Step 3: Recommend the optimal crops for planting given the current conditions.
            """,
            "prompt_chaining": [
                "Evaluate current water levels for the specified lat/long points.",
                "Analyze the impact of these water levels on tomato crop yield.",
                "Provide recommendations for optimal crop selection given the current conditions."
            ],
            "memory_agent_action": """
            Given historical water and nutrient data, predict the tomato crop yield for specific lat/long regions."
            """
        }
    elif user_role == "architect":
        return {
            "zero_shot": "Design a sustainable water and nutrient management plan for agricultural regions with specific Lat/Long points.",
            "few_shot": """
            Examples:
            - Implementing efficient irrigation systems in water-scarce areas led to significant improvements in the crop yield in Lat/Long points
            - Introducing advanced nutrient management techniques reduced wastage and increased soil fertility.
            For region Y, design a plan to optimize water and nutrient usage.
            """,
            
            "chain_of_thought": """
            Step 1: Assess the current water and nutrient infrastructure in the specified region.
            Step 2: Identify potential improvements in irrigation and nutrient distribution systems.
            Step 3: Propose a sustainable plan for long-term water and nutrient management.
            """,
            "prompt_chaining": [
                "Assess the current state of water and nutrient infrastructure for the specified lat/long points.",
                "Identify areas for improvement in irrigation and nutrient distribution.",
                "Design a comprehensive plan to optimize water and nutrient usage sustainably."
            ],
            "memory_agent_action": """
            Based on historical infrastructure data and current conditions, design a sustainable water and nutrient management plan for smart city"
            """
        }
    elif user_role == "planner":
        return {
            "zero_shot": "Plan the allocation of water and nutrients to maximize agricultural productivity in various in Lat/Long points.",
            "few_shot": """
            Examples:
            - Allocating more water to regions with high crop demands resulted in better yields.
            - Adjusting nutrient distribution based on soil quality led to more efficient use of resources.
            For region Z, how would you allocate water and nutrients to maximize productivity?
            """,
            "chain_of_thought": """
            Step 1: Review the current allocation of water and nutrients in the specified regions.
            Step 2: Analyze the productivity impact of the current allocation.
            Step 3: Recommend adjustments to optimize the distribution of resources.
            """,
            "prompt_chaining": [
                "Review the current allocation of water and nutrients for the specified lat/long points.",
                "Analyze the impact of the current resource allocation on productivity.",
                "Suggest optimal adjustments to improve resource distribution and maximize productivity."
            ],
            "memory_agent_action": """
            Using historical data on water and nutrient allocation, plan the distribution of resources for specific in Lat/Long points."
            """
        }
    else:
        return {}

#### Generative AI Architecture model for Agricuture Smart City
#### Class for Managing Prompts by Role
#### Class AgricultureAIRolesPrompts:
 Manages and displays prompts for different roles which represents clients in China for the Gen AI prompt engineering session. This class integrates functionality of the AgricultureAI class to execute and print results for prompt engineering AI tasks. There were also fine tune testing. See code on fine tuning.

In [9]:
class AgricultureAIRolesPrompts:
  
    def base_prompt(self, role):
        prompts = get_prompts(role)
        print(f"\n{role.capitalize()} Prompts:")
        
    def zero_shot_classification_prompt(self, role):
        zero_shot_result = agriculture_ai.zero_shot_classification(prompts["zero_shot"])
        print(f"Zero-shot classification result for {role}:\n{zero_shot_result}\n")
              
    def chain_of_thought_prompt(role):
        chain_of_thought_result = agriculture_ai.chain_of_thought(prompts["chain_of_thought"])
        print(f"Chain of thought result for {role}:\n{' '.join(chain_of_thought_result)}\n")
             
    def chaining_prompt(role):
        prompt_chaining_result = agriculture_ai.prompt_chaining(prompts["prompt_chaining"])
        print(f"Prompt chaining result for {role}:\n{' '.join(prompt_chaining_result)}\n")               
        
    def memory_agent_action_prompt(role):
        prompts = get_prompts(role)  # Ensure you have the prompts for the given role
        memory_buffer = "Historical data shows varying water and nutrient levels across different lat/long regions."
        memory_agent_action_result, action = agriculture_ai.memory_agent_action(prompts["few_shot"], memory_buffer)
        print(f"Memory agent action result for {role}:\n{memory_agent_action_result}\nAction taken:\n{action}\n")


#### Main Functionality for Running Prompts and Analysis:
#### main_prompts:
 Initializes the language model and runs various prompt-based analyses for different user roles. It also performs image analysis for smart city to be sustanable in land growth, water levels, and nutrient levels, and displays the results

In [10]:
def main_prompts():
    api_key = "your_openai_api_key_here"
    llm = instantiate_LLM("OpenAI", api_key, temperature=0.7, top_p=0.9, model_name="gpt-4")
    customer_roles = ["agriculture_manager", "architect", "planner"]
    for business_person_role in customer_roles:
        
        base_prompt(business_person_role)
        zero_shot_classification_prompt(business_person_role)
        chain_of_thought_prompt(business_person_role)
        chaining_prompt(business_person_role)
        memory_agent_action_prompt(business_person_role)
        
    example_image_path = "path_to_your_image.jpg"
    lat_long_points = lat_long_points_agriculture_coordinates()
    land_growth_image, NDVI = agriculture_ai.analyze_land_growth(example_image_path)
    cv2.imwrite("land_growth_analysis.jpg", land_growth_image)
    
    water_level_percentage, water_regions = agriculture_ai.detect_water_levels(example_image_path)
    print(f"Detected Water Level Percentage: {water_level_percentage}%")
    
    route_contours, route_count = agriculture_ai.vehicle_route_analysis(example_image_path)
    print(f"Number of detected vehicle routes: {route_count}")
    
    nutrient_levels_image, nutrient_mask = agriculture_ai.analyze_nutrient_levels(example_image_path)
    cv2.imwrite("nutrient_levels_analysis.jpg", nutrient_levels_image)
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.title("Land Growth (NDVI)")
    plt.imshow(cv2.cvtColor(land_growth_image, cv2.COLOR_BGR2RGB))
    
    plt.subplot(1, 3, 2)
    plt.title("Water Levels")
    plt.imshow(cv2.cvtColor(water_regions, cv2.COLOR_BGR2RGB))
    
    plt.subplot(1, 3, 3)
    plt.title("Nutrient Levels")
    plt.imshow(cv2.cvtColor(nutrient_levels_image, cv2.COLOR_BGR2RGB))
    
    plt.show()

#### Fine-Tuning Processes:
### function fine_tuning
  Outlines steps for fine-tuning the language model using different techniques (PEFT, LoRA, RLHF) and adjusting model responses based on feedback and specific instructional contexts
  
Fine tuning models were explored for prompt enginering performanc, the models test and implemented for this LLM Gen AI were evauated and tested: PEFT, LoRA, RLHF

This model utilized Parameter-Efficient Fine-Tuning (PEFT) method for LLM LangChain, also fine tune LLM for scalable LLM solutions using LoRA (Low-Rank Adaptation), and evaluated the Generative Agent Retrieval-Augmented Learning Human Feedback (RLHF) to ensure prompt engineering for performance was sufficient to move forward with the delivery.

Let me outline key recommendations in fine-tuning. The Parameter-Efficient Fine-Tuning (PEFT) method. This approach optimizes prompt engineering by efficiently adjusting prompt parameters to enhance model performance. By incorporating a small adapter into the pre-trained model, PEFT reduces the number of parameters required for fine-tuning, resulting in a more effective and computationally efficient model.

In [11]:
def fine_tuning():
    llm.fine_tune_peft(dataset)
    llm.fine_tune_lora(dataset)
    reward_model = llm.load_reward_model()
    llm.rl_fine_tuning(dataset, reward_model)
    prompt = "Explain the effects of climate change on agriculture."
    llm.prompt_tuning(prompt, context_window=128)
    llm.fine_tune_with_instructions(dataset, context_window=128)